<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 4. Сравнение алгоритмов классификации на наборах данных репозитория UCI</center>

В рамках данного задания нужно выбрать один из наборов данных для задачи классификации из [репозитория UCI](http://archive.ics.uci.edu/ml/), выбрать несколько алгоритмов классификации и сравнить их на этом наборе данных.

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%pylab inline
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


**Выберите любой набор данных для задачи классификации из [репозитория UCI](http://archive.ics.uci.edu/ml/) и загрузите его, используя функцию urlopen. Конечно, можно и скачать данные, а потом открыть csv-файл с помощью pandas.**

In [2]:
'''
Таким методом почему-то не грузилось. Но pandas.read_csv, согласно документации, это умеет делать сам:
 >>> The string could be a URL. Valid URL schemes include http, ftp, s3, and file.
'''
# import sys
# if sys.version_info.major == 2:
#     from urllib import urlopen
# elif sys.version_info.major == 3:
#     from urllib.request import urlopen
# raw_data = urlopen(url)
# data = np.loadtxt(raw_data, delimiter=",")


url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data'
'''
The data are generated to simulate registration of high energy gamma particles
in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique

Attribute Information:

1. fLength: continuous # major axis of ellipse [mm] 
2. fWidth: continuous # minor axis of ellipse [mm] 
3. fSize: continuous # 10-log of sum of content of all pixels [in #phot] 
4. fConc: continuous # ratio of sum of two highest pixels over fSize [ratio] 
5. fConc1: continuous # ratio of highest pixel over fSize [ratio] 
6. fAsym: continuous # distance from highest pixel to center, projected onto major axis [mm] 
7. fM3Long: continuous # 3rd root of third moment along major axis [mm] 
8. fM3Trans: continuous # 3rd root of third moment along minor axis [mm] 
9. fAlpha: continuous # angle of major axis with vector to origin [deg] 
10. fDist: continuous # distance from origin to center of ellipse [mm] 
11. class: g,h # gamma (signal), hadron (background) 
'''
import pandas as pd
features = [
    'fLength',
    'fWidth', 
    'fSize', 
    'fConc', 
    'fConc1', 
    'fAsym',
    'fM3Long',
    'fM3Trans',
    'fAlpha',
    'fDist',
    'class']
data = pd.read_csv(url, names=features)

**Выведите основные характеристики выборки (число объектов, признаков, число классов) и разделите выборку на обучение и контроль с помощью функции train_test_split**

In [3]:
from sklearn.model_selection import train_test_split

# Print basic data stats: 
# number of objects, number of features, number of classes 
print('Число объектов: {},'.format(data.shape[0]))
print('Число признаков: {},'.format(data.shape[1]))
print('Число классов: {}'.format(data['class'].nunique()))

# Extract objects and labels from data
y = data['class']
data.drop(['class'], axis=1, inplace=True)
X = data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Число объектов: 19020,
Число признаков: 11,
Число классов: 2


## 1. Классификаторы без настройки параметров 

**Выберите несколько (2-3) классификаторов из sklearn. Для удобства (чтобы не переписывать один и тот же код для каждого классификатора) можно создать список классификаторов, который, например, может состоять из пар вида (classifier_instance(), 'classifier name').**

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

classifiers = [
    (LogisticRegression(random_state=42), 'logit'),
    (KNeighborsClassifier(), 'knn'),
    (DecisionTreeClassifier(random_state=42), 'tree'),
#     (SVC(probability=True, random_state=42), 'svm')
]

**С помощью 5-кратной кросс-валидации оцените качество каждого классификатора на обучающей выборке. Используйте функцию cross_val_score. Попробуйте использовать разные метрики качества (параметр scoring)**

In [28]:
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

def mean_cross_val_score_by_scoring(classifier, X, y, cv, scoring):
    clf_scores = cross_val_score(classifier[0], X, y, cv=cv, scoring=scoring)
    return np.mean(clf_scores)    

n_folds = 5
scorings = ['accuracy', 'average_precision']

scores = dict(zip(scorings, [[] for _ in range(len(scorings))]))

skf = StratifiedKFold(n_splits=n_folds)

for product in itertools.product(classifiers, scorings):
    # Do it for each classifier
    mean = mean_cross_val_score_by_scoring(product[0], X_train, y_train, cv=skf, scoring=product[1])     
    scores[product[1]].append(mean)
# Print the name of classifier with
# the best score and the score
for scoring in scores:
    print('Метрика качества: {}'.format(scoring))
    print('\tЛучший классификатор: {}, со значением: {}'\
          .format(classifiers[np.argmax(scores[scoring])][1], max(scores[scoring])))

ValueError: Data is not binary and pos_label is not specified

In [ ]:
# from sklearn.model_selection import cross_val_score
# cross_val_score?
# print('Лучший классификатор: {}, со значением: {}'.format(scores[np.argmax(scores)][1], max(scores)))
scorings = ['accuracy', 'neg_log_loss', 'roc_auc']
for i in zip(scorings, [[] for _ in range(len(scorings))]):
    print(i)

# Print the name of classifier with
# the best score and the score
# for scoring in scores:
#     print('Метрика качества: {}'.format(scoring))
#     print('\tЛучший классификатор: {}, со значением: {}'\
#           .format(classifiers[np.argmax(scores[scoring])][1], max(scores[scoring])))

In [ ]:
'''
Так как cross_val_score выполняется на этом датасете для SVM ~ 1 мин, выделил в отдельнуя ячейку :)
Примерное выполнение ~ 3 мин.
'''
(SVC(probability=True, random_state=42), 'svm')

**Обучите те же классификаторы без параметров на всей обучающей выборке и проверьте их качество на тестовой выборке. Является ли лучшим классификатор, который имеет наибольшее качество на кросс-валидации?**

In [ ]:
from sklearn.metrics import accuracy_score

scores = []
for classifier in classifiers:
    # Do it for each classifier
    clf = # Ваш код здесь
    clf.fit # Ваш код здесь
    y_pred = clf.predict # Ваш код здесь
    clf_score = accuracy_score # Ваш код здесь
    scores.append(clf_score)

# Print the name of classifier with
# the best score and the score
print # Ваш код здесь

## 2. Классификаторы с настройкой параметров 

**Настройте параметры выбранных вами алгоритмов с помощью GridSearchCV и выберите лучший классификатор**

In [ ]:
from sklearn.model_selection import GridSearchCV 

n_folds = 5
scores = []

for classifier in classifiers:
    # Do it for each classifier
    clf_params = # Ваш код здесь
    grid = GridSearchCV # Ваш код здесь
    clf_best_score = # Ваш код здесь
    scores.append(clf_best_score)

# Print the name of classifier with
# the best score, the score and the params
print # Ваш код здесь

**Для лучшего классификатора и набора параметров для него выберите более мелкую сетку (для того чтобы уточнить лучшие значения параметров). Для этого классификатора настройте параметры по обучающей выборке с помощью GridSearchCV. Постройте график зависимости среднего качества на кросс-валидации от значений параметров**

In [ ]:
clf_params = # Ваш код здесь
grid = GridSearchCV # Ваш код здесь
clf_best_score = # Ваш код здесь
clf_best_params = # Ваш код здесь
clf_best = # Ваш код здесь

mean_validation_scores = # Ваш код здесь
plt.plot # Ваш код здесь

# Print the best parameters and score
print # Ваш код здесь

**Для полученной модели посчитайте качество на тестовой выборке и сравните его с качеством, полученным без настройки параметров**

In [ ]:
y_pred = clf_best.predict # Ваш код здесь
clf_score = accuracy_score # Ваш код здесь

# Print the score
print # Ваш код здесь

**Можно поварьировать параметр scoring и позапускать написанный код ещё раз. Изменится ли лучший классификатор?**

## Ссылки:

 - <a href="http://archive.ics.uci.edu/ml/">UCI Machine Learning Repository</a> 
 - <a href="http://scikit-learn.org/stable/index.html">Scikit-learn documentation</a>